In [ ]:
!pip install vllm pyngrok

import os
import subprocess
import time
import requests
from pyngrok import ngrok

# --- CONFIGURATION ---
# ⚠️ PASTE YOUR TOKEN BELOW inside the quotes
!ngrok authtoken "YOUR_NGROK_KEY_HERE"
MODEL_ID = "Qwen/Qwen2.5-3B-Instruct"  # <--- UPGRADED MODEL

# 1. Authenticate Ngrok
os.system(f"ngrok authtoken {NGROK_TOKEN}")

# 2. Start vLLM Server (Optimized for Colab T4)
print(f"🚀 Starting vLLM Server with {MODEL_ID}...")
print("⏳ Downloading model & loading weights... (This takes 3-5 mins)")

# Open a log file to capture output
log_file = open("vllm_logs.txt", "w")

server_process = subprocess.Popen(
    [
        "python", "-m", "vllm.entrypoints.openai.api_server",
        "--model", MODEL_ID,
        "--dtype", "half",
        "--port", "8000",
        "--trust-remote-code",
        # --- UPGRADED SETTINGS ---
        "--gpu-memory-utilization", "0.95",  # Increased to use full GPU power
        "--max-model-len", "8192",           # Increased 4x (Reads longer PDFs)
        "--enforce-eager"                    # Keeps RAM usage stable
    ],
    stdout=log_file,
    stderr=subprocess.STDOUT
)

# 3. SMART WAIT LOOP
started = False
for i in range(60): # Increased attempts because 3B model takes slightly longer to load
    try:
        response = requests.get("http://localhost:8000/v1/models")
        if response.status_code == 200:
            print("\n✅ vLLM Server is UP and READY!")
            started = True
            break
    except requests.exceptions.ConnectionError:
        print(f"   ...waiting for server to boot (Attempt {i+1}/60)")
        time.sleep(10)

# 4. Start Ngrok
if started:
    try:
        ngrok.kill()
        public_url = ngrok.connect(8000).public_url
        print(f"\n🎉 SUCCESS! API URL: {public_url}/v1")
        print("👉 Copy this URL to your Local VS Code.")

        # Keep cell alive
        try:
            while True:
                time.sleep(10)
        except KeyboardInterrupt:
            server_process.terminate()
            ngrok.kill()

    except Exception as e:
        print(f"❌ Ngrok Error: {e}")
else:
    print("\n❌ FAILED: Server did not start.")
    print("👇 SERVER LOGS (Why it failed):")
    print("="*50)

    log_file.close()
    try:
        with open("vllm_logs.txt", "r") as f:
            print(f.read())
    except FileNotFoundError:
        print("Error: Log file was not created.")

    print("="*50)
    server_process.terminate()

🚀 Starting vLLM Server with Qwen/Qwen2.5-3B-Instruct...
⏳ Downloading model & loading weights... (This takes 3-5 mins)
   ...waiting for server to boot (Attempt 1/60)
   ...waiting for server to boot (Attempt 2/60)
   ...waiting for server to boot (Attempt 3/60)
   ...waiting for server to boot (Attempt 4/60)
   ...waiting for server to boot (Attempt 5/60)
   ...waiting for server to boot (Attempt 6/60)
   ...waiting for server to boot (Attempt 7/60)
   ...waiting for server to boot (Attempt 8/60)

✅ vLLM Server is UP and READY!

🎉 SUCCESS! API URL: https://aa32fcc45afc.ngrok-free.app/v1
👉 Copy this URL to your Local VS Code.
